In [2]:
#!/usr/bin/python
#coding = utf-8
#Author: Selvaria
#国内主要炼厂各个化工产品产量
#来源：http://data.cpcifdata.org.cn/hggis/mainProdComp

from selenium import webdriver
import time

browser = webdriver.Chrome()
url_main = 'http://data.cpcifdata.org.cn/hggis/mainProdComp'
browser.get(url_main)
time.sleep(2)
elem_img = browser.find_element_by_xpath('//*[@id="provinceA"]/span')
browser.execute_script("arguments[0].click();", elem_img)
time.sleep(0.5)
oil_class = browser.find_element_by_xpath('//*[@id="provinceLiSub"]').text
#oil_class
product_list = oil_class.split('\n')
product_list = [i[2:] for i in product_list]
product_list

'A:ABS\nB:冰醋酸\nB:丙烯酸异辛酯\nB:苯胺\nB:丙烯\nB:苯乙烯\nB:苯酐\nB:丙酮\nB:丙烯酸乙酯\nB:丙二醇\nB:丙烯酸\nB:丙烯酸甲酯\nB:苯乙烯系热塑性弹性体（SBS）\nB:苯酚\nB:丙二醇甲醚\nB:丙烯腈\nB:丙烯酸丁酯\nC:醋酐\nC:醋酸丁酯\nC:纯碱\nC:醋酸仲丁酯\nC:粗苯\nC:纯苯\nC:醋酸乙酯\nC:纯MDI\nC:醋酸乙烯\nD:丁基橡胶\nD:丁酮\nD:丁苯橡胶\nD:涤纶长丝\nD:丁二烯橡胶\nD:电石\nD:丁腈橡胶\nD:对苯二甲酸二辛酯（DOTP）\nD:对二甲苯\nD:丁二烯\nD:涤纶短纤\nE:二乙二醇\nE:二甲苯\nE:二甲醚\nE:二甲基甲酰胺\nE:二氯甲烷\nE:EVA\nF:芳构化汽油\nF:复合肥\nH:合成氨\nH:环氧氯丙烷\nH:环己酮\nH:环氧树脂\nH:环氧乙烷\nH:黄磷\nJ:聚合MDI\nJ:己内酰胺\nJ:加氢苯\nJ:甲基异丁基酮（MIBK）\nJ:轿车轮胎\nJ:己二酸\nJ:聚乙烯醇\nJ:聚四氟乙烯\nJ:甲苯\nJ:聚苯乙烯\nJ:聚乙二醇\nJ:聚甲醛\nJ:聚酯切片\nJ:焦炭\nJ:甲醇\nJ:聚碳酸酯\nJ:甲醛\nJ:聚苯乙烯泡沫（EPS）\nJ:基础油\nJ:甲基丙烯酸甲酯\nL:磷矿石\nL:磷酸二氢钾\nL:磷酸\nL:磷酸氢钙\nL:硫酸铵\nL:磷酸二铵\nL:LNG\nL:邻二甲苯\nL:硫酸\nL:氯化钾\nL:磷酸一铵\nL:六偏磷酸钠\nL:氯化铵\nL:硫黄\nM:MTBE\nN:农用薄膜\nN:尿素\nP:PVC\nP:PE\nP:PP粉\nP:PTA\nP:PA6\nP:PET瓶级\nP:PMMA\nP:PA66\nP:PP\nR:软泡\nR:R22\nR:溶剂油\nS:三聚氰胺\nS:双氧水\nS:烧碱\nS:三氯氧磷\nS:三聚磷酸钠\nS:SAP\nS:顺酐\nS:三羟甲基丙烷\nS:生物柴油\nS:三氯化磷\nS:三元乙丙橡胶\nS:石油沥青\nS:双向拉伸聚丙烯薄膜(BOPP)\nS:双酚A\nS:双向拉伸聚酯薄膜(BOPET)\nS:石油焦\nS:石蜡\nT:碳酸锂\nT:炭黑\nT:钛白粉\nT:碳酸二甲酯\nT:TDI\nW:未拉伸聚丙烯薄膜(CPP)\nX:辛醇\nX:硝酸\

In [27]:
import requests
import json
import pymysql

class PetroCapacity(object):
    
    def __init__(self, chemi_name):
        self.chemi_name = chemi_name
        
    def get_data_list(self):
        header = {
            "User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'
            }
        params = {
                "flid" : 'MP', 
                "a2221" : self.chemi_name
                }    
        url_json = 'http://data.cpcifdata.org.cn/hggis/dataMainProdComp/findDataMainProdCompListForMap'
        response = requests.get(url_json, headers=header, params=params).text
        response_dump = json.loads(response, strict=False)
        chemi_list = response_dump['rows']
        return chemi_list
    
    def update_data(self):
        chemi_list = self.get_data_list()
        for prod in chemi_list:
            web_id = prod['a01']
            company_name = prod['a02']
            cover = prod['a077']
            flid = prod['flid']
            capacity = prod['a2223']
            unit = prod['a2222']
            region_code = prod['d2431']
            longitude = prod['x']
            latitude = prod['y']
            row = [web_id, company_name, flid, region_code, capacity, unit, cover, longitude, latitude]
            self.update_single(row)
    
    def update_single(self, row):
        connection = pymysql.connect(host = '47.92.25.70',user = 'root',password = 'Wfn031641',db = 'cxd_data',charset = 'utf8')
        row.append(self.chemi_name)
        with connection.cursor() as cursor: 
            sql_k = "insert into `petro_capacity_civil` (`web_id`,`com`,`flid`,`region_code`,`capacity`,`unit`,`cover`,`longitude`,`latitude`,`product`)\
            values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql_k,row)
            connection.commit()
        connection.close()
        
def main():
    for prod in product_list:
        pc = PetroCapacity(prod)
        pc.update_data()
        print('%s已完成' %prod)
        
if __name__ == "__main__":
    main()

D:\Selvaria\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'cover' at row 1")
  result = self._query(query)


ABS已完成
冰醋酸已完成
丙烯酸异辛酯已完成
苯胺已完成
丙烯已完成
苯乙烯已完成
苯酐已完成
丙酮已完成
丙烯酸乙酯已完成
丙二醇已完成
丙烯酸已完成
丙烯酸甲酯已完成
苯乙烯系热塑性弹性体（SBS）已完成
苯酚已完成
丙二醇甲醚已完成
丙烯腈已完成
丙烯酸丁酯已完成
醋酐已完成
醋酸丁酯已完成
纯碱已完成
醋酸仲丁酯已完成
粗苯已完成
纯苯已完成
醋酸乙酯已完成
纯MDI已完成
醋酸乙烯已完成
丁基橡胶已完成
丁酮已完成
丁苯橡胶已完成
涤纶长丝已完成
丁二烯橡胶已完成
电石已完成
丁腈橡胶已完成
对苯二甲酸二辛酯（DOTP）已完成
对二甲苯已完成
丁二烯已完成
涤纶短纤已完成
二乙二醇已完成
二甲苯已完成
二甲醚已完成
二甲基甲酰胺已完成
二氯甲烷已完成
EVA已完成
芳构化汽油已完成
复合肥已完成
合成氨已完成
环氧氯丙烷已完成
环己酮已完成
环氧树脂已完成
环氧乙烷已完成
黄磷已完成
聚合MDI已完成
己内酰胺已完成
加氢苯已完成
甲基异丁基酮（MIBK）已完成
轿车轮胎已完成
己二酸已完成
聚乙烯醇已完成
聚四氟乙烯已完成
甲苯已完成
聚苯乙烯已完成
聚乙二醇已完成
聚甲醛已完成
聚酯切片已完成
焦炭已完成
甲醇已完成
聚碳酸酯已完成
甲醛已完成
聚苯乙烯泡沫（EPS）已完成
基础油已完成
甲基丙烯酸甲酯已完成
磷矿石已完成
磷酸二氢钾已完成
磷酸已完成
磷酸氢钙已完成
硫酸铵已完成
磷酸二铵已完成


D:\Selvaria\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect integer value: '#N/A' for column 'region_code' at row 1")
  result = self._query(query)


LNG已完成
邻二甲苯已完成
硫酸已完成
氯化钾已完成
磷酸一铵已完成
六偏磷酸钠已完成
氯化铵已完成
硫黄已完成
MTBE已完成
农用薄膜已完成
尿素已完成
PVC已完成
PE已完成
PP粉已完成
PTA已完成
PA6已完成
PET瓶级已完成
PMMA已完成
PA66已完成
PP已完成
软泡已完成
R22已完成
溶剂油已完成
三聚氰胺已完成
双氧水已完成
烧碱已完成
三氯氧磷已完成
三聚磷酸钠已完成
SAP已完成
顺酐已完成
三羟甲基丙烷已完成
生物柴油已完成
三氯化磷已完成
三元乙丙橡胶已完成
石油沥青已完成
双向拉伸聚丙烯薄膜(BOPP)已完成
双酚A已完成
双向拉伸聚酯薄膜(BOPET)已完成
石油焦已完成
石蜡已完成
碳酸锂已完成
炭黑已完成
钛白粉已完成
碳酸二甲酯已完成
TDI已完成
未拉伸聚丙烯薄膜(CPP)已完成
辛醇已完成
硝酸已完成
新戊二醇已完成
溴素已完成
异戊二烯橡胶已完成
乙醇已完成
有机硅中间体已完成
硬泡已完成
乙烯已完成
异丁烯已完成
异丁醇已完成
异丁醛已完成
乙二醇已完成
乙腈已完成
原盐已完成
异丙醇已完成
亚磷酸已完成
乙醇胺已完成
1,4-丁二醇已完成
正丁醇已完成
载重汽车轮胎已完成


In [19]:
import requests
import json

header = {
    "User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'
    }
params = {
        "flid" : 'MP', 
        "a2221" : '乙烯'
        }    
url_json = 'http://data.cpcifdata.org.cn/hggis/dataMainProdComp/findDataMainProdCompListForMap'

response = requests.get(url_json, headers=header, params=params).text

json_dump = json.loads(response, strict=False)
json_dump 

{'rows': [{'a01': '053283024',
   'a02': '南京扬子石油化工有限责任公司',
   'a07': '',
   'a077': '合成树脂及塑料的生产与销售。（依法须经批准的项目，经相关部门批准后方可开展经营活动）合成树脂及塑料的生产与销售。（依法须经批准的项目，经相关部门批准后方可开展经营活动）详情',
   'a2222': '千吨',
   'a2223': '650',
   'd223': '',
   'd2431': '32',
   'd2432': '',
   'd2433': '',
   'flid': 'MP',
   'gudong1': '',
   'gudong2': '',
   'x': '118.800287713668',
   'y': '32.2840977434268'},
  {'a01': '070460504',
   'a02': '山西焦煤集团飞虹化工股份有限公司',
   'a07': '',
   'a077': '经营本企业自产产品；货物或技术进出口：经营本企业生产所需的原辅材料、仪器仪表、机械设备零配件及技术进口（国家禁止或涉及行政审批的除外）；承揽化工设备和零部件加工制作；本企业生产所需的原辅材料、仪器仪表、机械设备及零配件的检修、防腐保温及租赁服务；煤化工信息与技术咨询。（依法须经批准的项目，经相关部门批准后方可开展经营活动）经营本企业自产产品；货物或技术进出口：经营本企业生产所需的原辅材料、仪器仪表、机械设备零配件及技术进口（国家禁止或涉及行政审批的除外）；承揽化工设备和零部件加工制作；本企业生产所需的原辅\x00？？？\x18？癥Eof\x00\x00eNext\x00\x00\x01\x00？ 儀$\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00一眄\x00\x00區\x1f\x05\x00\x00\x18？\x18慄相飦？？\x18？\x00\x00\x00？？？？罹相？盹？\x18',
   'a2222': '千吨',
   'a2223': '300',
   'd223': '',
   'd2431': '14',
   'd2432': '',
   'd2433': '',

In [20]:
chemi_list = json_dump['rows']
len(chemi_list)

44

In [21]:
for prod in chemi_list:
    web_id = prod['a01']
    company_name = prod['a02']
    cover = prod['a077']
    flid = prod['flid']
    capacity = prod['a2223']
    unit = prod['a2222']
    region_code = prod['d2431']
    longitude = prod['x']
    latitude = prod['y']
    row = [web_id, company_name, flid, region_code, capacity, unit, cover, longitude, latitude]
    print(row)

['053283024', '南京扬子石油化工有限责任公司', '合成树脂及塑料的生产与销售。（依法须经批准的项目，经相关部门批准后方可开展经营活动）合成树脂及塑料的生产与销售。（依法须经批准的项目，经相关部门批准后方可开展经营活动）详情', 'MP', '650', '千吨', '118.800287713668', '32.2840977434268']
['070460504', '山西焦煤集团飞虹化工股份有限公司', '经营本企业自产产品；货物或技术进出口：经营本企业生产所需的原辅材料、仪器仪表、机械设备零配件及技术进口（国家禁止或涉及行政审批的除外）；承揽化工设备和零部件加工制作；本企业生产所需的原辅材料、仪器仪表、机械设备及零配件的检修、防腐保温及租赁服务；煤化工信息与技术咨询。（依法须经批准的项目，经相关部门批准后方可开展经营活动）经营本企业自产产品；货物或技术进出口：经营本企业生产所需的原辅材料、仪器仪表、机械设备零配件及技术进口（国家禁止或涉及行政审批的除外）；承揽化工设备和零部件加工制作；本企业生产所需的原辅\x00？？？\x18？癥Eof\x00\x00eNext\x00\x00\x01\x00？ 儀$\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00一眄\x00\x00區\x1f\x05\x00\x00\x18？\x18慄相飦？？\x18？\x00\x00\x00？？？？罹相？盹？\x18', 'MP', '300', '千吨', '111.804846113488', '36.2987141582511']
['078394943', '神华包头煤化工有限责任公司', '以煤为原料，生产、销售甲醇、甲醇制烯烃、聚丙烯、聚乙烯、硫磺、C4、C5、CO2、丁烯-1、乙烯、丙烯、丙烷、甲基叔丁基醚。热力生产及供应、工业用水、发电；对外贸易经营；相关技术服务。（在许可证有效期内经营）。 一般经营项目：无', 'MP', '300', '千吨', '109.949197459195', '40.6272022782745']
['078394943', '神华包头煤化工有限责任公司', '许可经营项目：以煤为原料，生产、销售甲醇、甲醇制烯烃、聚丙烯、聚乙烯、硫磺、C4、C5、CO2、丁烯-